In [ ]:
import csv
import pandas as pd

In [ ]:
#Loads all datafiles into dataframes of the same name.  Not sure the with open statements are necessary.
filenames = ['colors.csv', 'inventories.csv', 'inventory_parts.csv', 'inventory_sets.csv', 'part_categories.csv', 'parts.csv', 'sets.csv', 'themes.csv']

with open(filenames[0]) as datafile:
    colors = pd.read_csv(datafile)
with open(filenames[1]) as datafile:
    inventories = pd.read_csv(datafile)
with open(filenames[2]) as datafile:
    inventory_parts = pd.read_csv(datafile)
with open(filenames[3]) as datafile:
    inventory_sets = pd.read_csv(datafile)
with open(filenames[4]) as datafile:
    part_categories = pd.read_csv(datafile)
with open(filenames[5]) as datafile:
    parts = pd.read_csv(datafile)
with open(filenames[6]) as datafile:
    sets = pd.read_csv(datafile)
with open(filenames[7]) as datafile:
    themes = pd.read_csv(datafile)

In [ ]:
#Add theme info to rows in sets dataframe
sets = pd.merge(sets, themes, left_on = 'theme_id', right_on = 'id')
sets = sets.drop('id', axis = 1) #extra column from merge
sets = sets.set_index(['set_num']) #rename the index to the set id
sets.columns = ['setname', 'year', 'theme_id', 'num_parts', 'theme_name', 'parent_id'] #slightly inefficient way to rename the name column

In [ ]:
inventory_parts = pd.merge(inventory_parts, colors, left_on = 'color_id', right_on = 'id')

In [ ]:
inventory_parts = inventory_parts.drop('id', axis = 1)
inventory_parts.columns=['inventory_id', 'part_num', 'color_id', 'quantity', 'is_spare', 'color_name', 'rgb', 'is_trans']

In [ ]:
inventories = inventories.set_index('id')

In [ ]:
#Attempt to add the number of parts of each color to the inventories dataframe.
#This takes a very long time.
inventoryids = inventory_parts['inventory_id'].unique() #Gets the unique list of inventory_parts inventory ids
colornames = colors['name'].unique() #same for color ids
#initialize color column to 0
for name in colornames:
    inventories[name]=0
groupedinventory = inventory_parts.groupby('inventory_id') #Generates a collection of little dfs, one for each id
for k in inventoryids:
    colorgroups = groupedinventory.get_group(k).groupby('color_name') #groups each little df by color_name
    #counts the number of parts of each color, and inserts it to the appropriate entry in inventory sets
    for name in colornames:
        try: 
            inventories.loc[k,name] = int(colorgroups.get_group(name)['quantity'].sum())
        except KeyError:
            pass #have the code ignore all KeyErrors and continue

In [ ]:
inventories

This could now be merged with the sets dataframe and it would be sufficient, but there's something really weird going on with this data that needs to be explored more:

Run the 2nd and 3rd cells in this notebook again before running these, otherwise you'll get a bunch of keyerrors.

In [ ]:
#all of the entires in inventories have unique inventory ids - no inventory shows up in more than one set
#some sets do have more than one inventory
print(len(inventories))
print(len(inventories['id'].unique())) #I set id to the index above, whoops.  Reload the data from the begining and uncomment
print(len(inventories['set_num'].unique()))

In [ ]:
#inventory_sets has inventories that are in more than one set, not very many over all
print len(inventory_sets)
print len(inventory_sets['set_num'].unique())
print len(inventory_sets['inventory_id'].unique())
print len(inventory_parts['inventory_id'].unique())

In [ ]:
#the total number of sets is slightly larger than the number of unique sets showing up in the inventories df
print(len(sets))
print(len(inventories['set_num'].unique()))

In [ ]:
#There also appear to be inventories that have no entries in inventory_parts - no parts in those?
print len(inventory_parts)
print 'unique inventories in inventory_parts: ', len(inventory_parts['inventory_id'].unique())
print 'unique inventories in inventories: ', len(inventories['id'].unique())
print 'unique inventories in inventories_sets: ', len(inventory_sets['inventory_id'].unique())

In [ ]:
#all of the parts and sets entries are in inventories:
ip = set(inventory_parts['inventory_id'].unique())
i_s = set(inventory_sets['inventory_id'].unique())
i = set(inventories['id'].unique())
print len(i.difference(ip)) #number of things in i that aren't in ip
print len(inventories['id'].unique()) - len(inventory_parts['inventory_id'].unique())
print len(i.difference(i_s))
print len(inventories['id'].unique()) - len(inventory_sets['inventory_id'].unique())

In [ ]:
#This prints out all the set information for all the inventories that don't entries in inventory_parts
#A lot of it is swag, but some things list that they have parts!
inventory_parts
ip = set(inventory_parts['inventory_id'].unique())
i = set(inventories['id'].unique())

inventories_not_in_parts = i.difference(ip)
#print inventories_not_in_parts

for x in inventories_not_in_parts:
    weird_inv = inventories[inventories['id'] == x]
    print sets.loc[weird_inv.iloc[0]['set_num']]
